In [1]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [2]:
%cd slt

/content/slt


In [3]:
!chmod +x data/download.sh
!data/download.sh

--2020-12-27 21:18:21--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  18.0MB/s    in 99s     

2020-12-27 21:20:00 (18.6 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2020-12-27 21:20:00--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  18.9MB/s    in 7.6s    

2020-12-27 21:20:09 (16.4 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [6]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 9.5MB/s 
     |████████████████████████████████| 112kB 10.0MB/s 
     |████████████████████████████████| 163kB 19.3MB/s 
     |████████████████████████████████| 399kB 23.8MB/s 
     |████████████████████████████████| 2.3MB 35.9MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 2.4MB 63.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 296kB 56.8MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 13.1MB 60.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |███

In [7]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!cp -r ../drive/MyDrive/slt/BOVW/test_bovw_features.zip data/test_bovw_features.zip
!cp -r ../drive/MyDrive/slt/BOVW/dev_bovw_features.zip data/dev_bovw_features.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part1.zip data/train_bovw_features_part1.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part2.zip data/train_bovw_features_part2.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part3.zip data/train_bovw_features_part3.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part4.zip data/train_bovw_features_part4.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part5.zip data/train_bovw_features_part5.zip
!cp -r ../drive/MyDrive/slt/BOVW/train_bovw_features_part6.zip data/train_bovw_features_part6.zip


!unzip -q data/test_bovw_features.zip -d data
!unzip -q data/dev_bovw_features.zip -d data
!unzip -q data/train_bovw_features_part1.zip -d data
!unzip -q data/train_bovw_features_part2.zip -d data
!unzip -q data/train_bovw_features_part3.zip -d data
!unzip -q data/train_bovw_features_part4.zip -d data
!unzip -q data/train_bovw_features_part5.zip -d data
!unzip -q data/train_bovw_features_part6.zip -d data

In [8]:
!mv data/data/dev_features256 data/dev
!mv data/data/train_features256 data/train
!mv data/data/test_features256 data/test

In [9]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:25<00:00, 281.21it/s]


In [10]:
!python -m signjoey train configs/sign.yaml

2020-12-27 21:29:47.141853: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-27 21:29:53.634140: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-27 21:29:53.638341: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-27 21:29:53.692272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-27 21:29:53.692965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.75GiB deviceMemoryBandwidth: 836.37GiB/s
2020-12-27 21:29:53.693006: I tensorflow/stream_executor/platform/defau

In [11]:
!cp -r ./sign_sample_model "../drive/MyDrive/slt/models/bovw" 